# Markov Chains
A Markov chain model assumes that the change between match states does not depend on what has happened earlier, only on the current state. If the attacking team has the ball in midfield, the probability of it going into the box is the same, irrespective of whether the team has carried out a long sequence of passes, or if the ball has just arrived from a punt. This assumption oversimplifies aspects of football tactics, but it is a reasonable starting point.  Generally the window associated with ball movement is limited to about 10sec.

In this section expected threat will be determined in terms of a Markov chain.

In [1]:
import numpy as np

### Setting up the matrix
Set up the pass matrix A and the goal vector g.  The two arrays will represent the probability of the ball moving into anther area of the field (matrix A) and the likelihood of a goal from the associated position (vector g).  This will be used to solve for xT.

Table of probabilites:

|from/to|Mid|Box|Wing|Goal|
|---|---|---|---|---|
|Mid|25%|20%|10%|5%|
|Box|10%|25%|20%|15%|
|Wing|10%|10%|25%|5%|


In [2]:
# Pass matrix
A = np.matrix([[0.25, 0.20, 0.1], [0.1, 0.25, 0.2],[0.1, 0.1, 0.25]])
A

matrix([[0.25, 0.2 , 0.1 ],
        [0.1 , 0.25, 0.2 ],
        [0.1 , 0.1 , 0.25]])

In [3]:
# Goal vector
g = np.transpose(np.matrix([0.05, 0.15, 0.05]))
g

matrix([[0.05],
        [0.15],
        [0.05]])

### Linear algebra method
Solve the linear algerbra formula $(I-A)xT = g$ for xT.

In [4]:
xT1 = np.linalg.solve(np.identity(3) - A,g)

print('Expected Threat')
print('Central, Box, Wing')
print(np.transpose(xT1))

Expected Threat
Central, Box, Wing
[[0.14991763 0.25205931 0.12026359]]


### Iterative method
Iterate xT’ = A xT + g to update through each move of the ball.

In [8]:
xT2=np.zeros((3,1))
for t in range(10):
#    print(np.matmul(A,xT2) + g)
   xT2 = np.matmul(A,xT2) + g

In [9]:
print('Expected Threat')
print('Central, Box, Wing')
print(np.transpose(xT2))

Expected Threat
Central, Box, Wing
[[0.14966911 0.25182476 0.12007973]]


### Simulation method
Simulate num_sim possessions, starting from each of the three areas.  The simulation uses a random float generation to evaluate the 10 simulations and determine if a goal was scored.  The expected threat is then recalculated based on the number of goals ratio over the number of simulations conducted.  This will vary with each run through the code.  I provides an indication of how this might be evaluated manually.

Originally this was run at 10 simulations and it is quite vairable.  As you increase the simulations to 100, the values begin to coalesce at the same numbers in the prior solutions.

In [20]:
num_sims=100
xT3=np.zeros(3)

description = {0: 'Central', 1: 'Wing', 2: 'Box' }

for i in range(3):
    num_goals = 0

    print('---------------')
    print('Start from ' + description[i] )
    print('---------------')

    for n in range(num_sims):

        ballinplay=True
        #Initial state is i
        s = i
        describe_possession=''

        while ballinplay:
            r=np.random.rand()

            # Make commentary text
            describe_possession = describe_possession + ' - ' + description[s]


            #Cumulative sum of in play probabilities
            c_sum=np.cumsum(A[s,:])
            new_s = np.sum(r>c_sum)
            if new_s>2:
                #Ball is either goal or out of play
                ballinplay=False
                if r < g[s] + c_sum[0,2]:
                    #Its a goal!
                    num_goals = num_goals + 1
                    describe_possession = describe_possession + ' - Goal!'
                else:
                    describe_possession = describe_possession + ' - Out of play'
            s = new_s

        print(describe_possession)

    xT3[i] = num_goals/num_sims


print('\n\n---------------')
print('Expected Threat')
print('Central, Box, Wing')
print(xT3)

---------------
Start from Central
---------------
 - Central - Goal!
 - Central - Wing - Goal!
 - Central - Out of play
 - Central - Wing - Out of play
 - Central - Wing - Out of play
 - Central - Out of play
 - Central - Central - Wing - Out of play
 - Central - Central - Box - Central - Central - Box - Out of play
 - Central - Wing - Goal!
 - Central - Box - Central - Out of play
 - Central - Out of play
 - Central - Central - Wing - Wing - Goal!
 - Central - Goal!
 - Central - Out of play
 - Central - Central - Central - Box - Box - Box - Out of play
 - Central - Out of play
 - Central - Wing - Goal!
 - Central - Out of play
 - Central - Out of play
 - Central - Wing - Box - Wing - Wing - Box - Goal!
 - Central - Central - Out of play
 - Central - Central - Goal!
 - Central - Central - Wing - Box - Out of play
 - Central - Central - Box - Out of play
 - Central - Out of play
 - Central - Out of play
 - Central - Central - Out of play
 - Central - Central - Central - Out of play
 - 